In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans



In [4]:
crsp_data = pd.read_csv("data/crsp_data/crsp_data_1962-01-01_1962-12-31.csv")
df = pd.read_csv("data/cum_returns/cum_returns_index_1962-01-01_1962-12-31.csv", index_col=0, parse_dates=True)
X = df.T 

In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=10)  # keep top 10 factors
X_pca = pca.fit_transform(X_scaled)


In [13]:

kmeans = KMeans(n_clusters=20, random_state=0)
labels = kmeans.fit_predict(X_pca)

# Assign cluster labels back to each stock
clusters = pd.Series(labels, index=X.index, name='cluster')



In [21]:
for cluster_id, members in clusters.groupby(clusters):
    print(f"\nCluster {cluster_id}:")
    print(members.index.tolist())


Cluster 0:
['10671.0', '10727.0', '12554.0', '14488.0', '14955.0', '15341.0', '15544.0', '15798.0', '17101.0', '17435.0', '18606.0', '18948.0', '19027.0', '19414.0', '20327.0', '21864.0', '22904.0', '23296.0', '25486.0', '26331.0', '26569.0', '53023.0']

Cluster 1:
['10188.0', '10524.0', '10532.0', '11033.0', '12044.0', '13434.0', '14154.0', '16512.0', '16643.0', '16651.0', '16707.0', '16731.0', '17400.0', '17689.0', '17734.0', '18390.0', '18569.0', '19203.0', '19289.0', '19561.0', '19617.0', '19684.0', '20597.0', '21733.0', '21813.0', '21901.0', '22242.0', '22330.0', '22488.0', '22568.0', '23050.0', '23288.0', '23472.0', '24205.0', '24512.0', '24651.0', '24897.0', '24993.0', '25427.0', '25769.0', '26008.0', '26200.0', '26251.0', '26374.0', '26403.0', '26956.0', '26980.0', '27051.0', '27123.0', '27190.0', '27270.0', '27430.0', '27529.0', '27748.0', '27916.0', '27991.0', '28097.0', '28169.0']

Cluster 2:
['10014.0', '10102.0', '10209.0', '10364.0', '10401.0', '10495.0', '10575.0', '111

In [18]:
from pairs_formation import compute_pairwise_ssd, pair_matching, get_stock

In [20]:
pairs_ml = []

for cluster_id, members in clusters.groupby(clusters):
    stocks = members.index.values
    if len(stocks) < 2:
        continue

    sub = X.query("index.isin(@stocks)").T

    # Compute SSDs 
    print("Computing pairwise SSD")
    ssd_df = compute_pairwise_ssd(sub)

    # Match into pairs
    print("Matching into pairs")
    matched_pairs = pair_matching(ssd_df, crsp_data, max_pairs=10000)
    matched_pairs = matched_pairs.sort_values('ssd').reset_index(drop=True)
    matched_pairs_named = get_stock(matched_pairs)

    print(f"Total pairs formed: {len(matched_pairs_named)} for cluster ID: {cluster_id} with {len(stocks)} stocks")
    print(matched_pairs_named)
    matched_pairs_named.to_csv("ml_pairs/total_matched_pairs/matched_pairs_1963-01-01_1963-12-31_cluster_"+str(cluster_id)+".csv", index=False)


Computing pairwise SSD
Matching into pairs
Loading library list...
Done
Total pairs formed: 11 for cluster ID: 0 with 22 stocks
    permno_1  permno_2       ssd                     comnam_1  permco_1  \
0      18948     53023  0.553154  ROCKWELL INTERNATIONAL CORP     21524   
1      18606     19414  0.564067                    AVCO CORP     22861   
2      14955     17435  0.680026     CHIQUITA BRANDS INTL INC     22568   
3      10727     20327  0.761742             BAYUK CIGARS INC     22219   
4      19027     26569  0.850450                CELANESE CORP     22888   
5      15341     15798  1.009400    WESTINGHOUSE AIR BRAKE CO     22600   
6      10671     23296  1.199569   BALDWIN LIMA HAMILTON CORP     22215   
7      14488     21864  2.148806           SOUTH P R SUGAR CO     22528   
8      15544     26331  2.432718                 MCCRORY CORP     22614   
9      17101     25486  4.137755                WILCOX OIL CO     22746   
10     12554     22904  4.585677  INTERNATIONAL

OperationalError: (psycopg2.OperationalError) connection to server at "wrds-pgdata.wharton.upenn.edu" (165.123.60.118), port 9737 failed: FATAL:  too many connections for role "sohrac"

(Background on this error at: https://sqlalche.me/e/20/e3q8)